In [1]:
import pandas as pd
from neuralprophet import NeuralProphet, set_log_level
import plotly.express as px
set_log_level("ERROR")

In [10]:
## df = pd.concat(map(pd.read_csv, ['../../data/train.csv', '../../data/data_merged.csv'])) #index 높이가 달라서 NaN으로 입력됨

#https://bigwaveai.tistory.com/28 참고
#https://colab.research.google.com/drive/1BLyPyWe9RMm4v5oy8f_RMXeKKdjl0vb0?usp=sharing#scrollTo=7oUK6ovdOpOQ

In [2]:
data=pd.read_csv("../../data/data_merged_with_train.csv", header=1)

In [3]:
data.head()

,date,air,real_pop,pop,pop_male,pop_female,mvpop,covid_total,covid_new,new_user,...,rain(mm),relative_humidity,distance_stage,distance_total,distance_value,distance_nominal,GW_demand,DDM_demand,SD_demand,JL_demand
0,1/1/2018,2124.0000,421.214,357785,161143,174428,260642,0,0,19.016,...,0.0,39.1,0.0,0,0.0,0,0.592,0.368,0.580,0.162
1,1/2/2018,144880.6667,408.056,357785,161143,174428,438460,0,0,19.016,...,0.0,42.0,0.0,0,0.0,0,0.840,0.614,1.034,0.260
2,1/3/2018,114977.0000,406.174,357785,161143,174428,444544,0,0,19.016,...,0.0,42.3,0.0,0,0.0,0,0.828,0.576,0.952,0.288
3,1/4/2018,2216.0000,409.168,357785,161143,174428,446740,0,0,19.016,...,0.0,43.0,0.0,0,0.0,0,0.792,0.542,0.914,0.292
4,1/5/2018,3282.0000,408.793,357785,161143,174428,478999,0,0,19.016,...,0.0,48.4,0.0,0,0.0,0,0.818,0.602,0.994,0.308


In [5]:
data.GW_demand.describe()

count    1461.000000
mean        6.209194
std         3.531408
min         0.000000
25%         3.256000
50%         5.740000
75%         9.444000
max        15.184000
Name: GW_demand, dtype: float64

In [6]:
data.columns

Index(['date', 'air', 'real_pop', 'pop', 'pop_male', 'pop_female', 'mvpop',
       'covid_total', 'covid_new', 'new_user', 'station', 'air.1',
       'real_pop.1', 'pop.1', 'pop_male.1', 'pop_female.1', 'mvpop.1',
       'covid_total.1', 'covid_new.1', 'new_user.1', 'station.1', 'air.2',
       'real_pop.2', 'pop.2', 'pop_male.2', 'pop_female.2', 'mvpop.2',
       'covid_total.2', 'covid_new.2', 'new_user.2', 'station.2', 'air.3',
       'real_pop.3', 'pop.3', 'pop_male.3', 'pop_female.3', 'mvpop.3',
       'covid_total.3', 'covid_new.3', 'new_user.3', 'station.3', '다이어트', '운동',
       '헬시플레저', '환경', '그린', '대기오염', '미세먼지', 'temp_avg', 'temp_low',
       'temp_high', 'rain(mm)', 'relative_humidity', 'distance_stage',
       'distance_total', 'distance_value', 'distance_nominal', 'GW_demand',
       'DDM_demand', 'SD_demand', 'JL_demand'],
      dtype='object')

In [10]:
data=data.rename(columns={"date":"ds", "GW_demand":"y"})

In [11]:
x_col_lst=['air', 'real_pop', 'mvpop', 'new_user', 'rain(mm)', 'relative_humidity', 'distance_nominal']
#y변수 정의
y_col_lst=['y']

In [12]:
X=data[x_col_lst]
Y=data[y_col_lst]

In [13]:
#autocorrelation을 잡기 위해서// lag을 몇 개 만들지?
#X변수들에 대한 lag 데이터 생성 (2 lag 까지만)
X_diff=X.diff()
X_diff.columns=X.columns+'_diff'

X_diff2=X.diff(periods=2)
X_diff2.columns=X.columns+'_diff2'

#Y변수들에 대한 lag 데이터 생성 (2 lag 까지만)
Y_diff=Y.diff()
Y_diff.columns=Y.columns+'_diff'

Y_diff2=Y.diff(periods=2)
Y_diff2.columns=Y.columns+'_diff2'

#X데이터 결합
X=pd.concat([X,X_diff,X_diff2],axis=1)

#Y데이터 결합
Y=pd.concat([Y,Y_diff,Y_diff2],axis=1)

#lag 데이터를 사용하므로 1행 ~ 2행 제거
#X=X.drop([0,1],axis=0)
#Y=Y.drop([0,1],axis=0)

#전체 데이터셋 생성
data_pret=pd.concat([data['ds'][2:],X,Y],axis=1)
data_pret=data_pret.reset_index(drop=True)
data_pret

,ds,air,real_pop,mvpop,new_user,rain(mm),relative_humidity,distance_nominal,air_diff,real_pop_diff,...,air_diff2,real_pop_diff2,mvpop_diff2,new_user_diff2,rain(mm)_diff2,relative_humidity_diff2,distance_nominal_diff2,y,y_diff,y_diff2
0,1/3/2018,114977.0000,406.174,444544,19.016,0.0,42.3,0,-29903.6667,-1.882,...,112853.0000,-15.040,183902.0,0.0,0.0,3.2,0.0,0.828,-0.012,0.236
1,1/4/2018,2216.0000,409.168,446740,19.016,0.0,43.0,0,-112761.0000,2.994,...,-142664.6667,1.112,8280.0,0.0,0.0,1.0,0.0,0.792,-0.036,-0.048
2,1/5/2018,3282.0000,408.793,478999,19.016,0.0,48.4,0,1066.0000,-0.375,...,-111695.0000,2.619,34455.0,0.0,0.0,6.1,0.0,0.818,0.026,-0.010
3,1/6/2018,2725.0000,426.190,403019,19.016,0.0,45.8,0,-557.0000,17.397,...,509.0000,17.022,-43721.0,0.0,0.0,2.8,0.0,0.618,-0.200,-0.174
4,1/7/2018,228553.3333,424.531,291662,19.016,0.0,42.1,0,225828.3333,-1.659,...,225271.3333,15.738,-187337.0,0.0,0.0,-6.3,0.0,0.598,-0.020,-0.220
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,12/29/2021,54.0000,372.609,359176,164.958,0.2,72.9,0,12.0000,-0.231,...,31.0000,-0.752,27014.0,0.0,0.2,12.0,0.0,4.490,-0.020,0.660
1457,12/30/2021,26.0000,375.444,377097,164.958,0.0,48.5,0,-28.0000,2.835,...,-16.0000,2.604,29655.0,0.0,0.0,-25.3,0.0,4.444,-0.046,-0.066
1458,12/31/2021,21.0000,373.701,347905,164.958,0.0,35.9,0,-5.0000,-1.743,...,-33.0000,1.092,-11271.0,0.0,-0.2,-37.0,0.0,3.616,-0.828,-0.874
1459,NaN,2124.0000,421.214,260642,19.016,0.0,39.1,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.592,NaN,NaN


In [14]:
#train test split
cutoff = "2021-01-01" #데이터 분할 기준
train = data_pret[data_pret['ds']<cutoff]
test = data_pret[data_pret['ds']>=cutoff]

In [15]:
#최종독립변수 이름 생성
col_lst=data_pret.columns
col_lst=col_lst.drop(['ds','y'])
col_lst=list(col_lst)
col_lst

['air',
 'real_pop',
 'mvpop',
 'new_user',
 'rain(mm)',
 'relative_humidity',
 'distance_nominal',
 'air_diff',
 'real_pop_diff',
 'mvpop_diff',
 'new_user_diff',
 'rain(mm)_diff',
 'relative_humidity_diff',
 'distance_nominal_diff',
 'air_diff2',
 'real_pop_diff2',
 'mvpop_diff2',
 'new_user_diff2',
 'rain(mm)_diff2',
 'relative_humidity_diff2',
 'distance_nominal_diff2',
 'y_diff',
 'y_diff2']

In [18]:
m = NeuralProphet(

growth='linear', # 추세 유형 설정(linear, discontinuous, off 중 선택 가능)

yearly_seasonality=True, #년간 계절성 설정

weekly_seasonality=True, #주간 계절성 설정

daily_seasonality=True, #일간 계절성 설정

batch_size=32,#배치 사이즈 설정

epochs=30,#학습 횟수 설정

learning_rate=0.1, # 학습률 설정

n_lags= 2, #lag를 2까지 사용하였으므로, lag를 2로 설정

)

#독립 변인(변수) 추가 및 정규화
m = m.add_lagged_regressor(names=col_lst, normalize="minmax") 

#학습 수행
metrics = m.fit(train, freq='h', validation_df=test, progress='plot')

ValueError: Inputs/targets with missing values detected. Please either adjust imputation parameters, or set 'drop_missing' to True to drop those samples.